In [ ]:
# install https://github.com/MarkusHaak/dlomix/ with pip
# OR uncomment to insert its path with sys:
#import os, sys
#sys.path.insert(0, os.path.abspath('../../dlomix/'))

In [ ]:
# set global seeds for reproducibility
from dlomix.utils import set_global_seed
set_global_seed(42)

In [ ]:
import numpy as np
import pandas as pd
import dlomix
from dlomix import constants, data, eval, layers, models, pipelines, reports, utils
import traceback
from dlomix.data import RetentionTimeDataset
import pickle
import os
from dlomix.models import PrositRetentionTimePredictor
from dlomix.reports import RetentionTimeReport
from matplotlib import pyplot as plt
from scipy.stats import ks_2samp
import tensorflow as tf
from dlomix.losses.quantile import QuantileLoss
from dlomix.eval.interval_conformal import IntervalSize, AbsoluteIntervalSize, RelativeCentralDistance, \
                                           IntervalConformalScore, IntervalConformalQuantile
from scipy.stats import ks_2samp

In [ ]:
# alphabet using the same PTM identifiers as in the created datasets
ALPHABET_MOD = {
    "A": 1,
    "C": 2,
    "D": 3,
    "E": 4,
    "F": 5,
    "G": 6,
    "H": 7,
    "I": 8,
    "K": 9,
    "L": 10,
    "M": 11,
    "N": 12,
    "P": 13,
    "Q": 14,
    "R": 15,
    "S": 16,
    "T": 17,
    "V": 18,
    "W": 19,
    "Y": 20,
    "^": 21,
    "}": 22,
}

# Quantile Regression

Run Quantile regression on (modified) baseline models.

In [ ]:
# dictionary storing the configuration of all models that shall be testet under their given label
models = {
    'PRT_med_<-1>:2' : {'config':['PRT_median', -1, tf.keras.Sequential([tf.keras.layers.Dense(2)])]},
    #'PRT_med_<-1>:32:2' : {'config':['PRT_median', -1, tf.keras.Sequential([tf.keras.layers.Dense(32, activation="relu"),tf.keras.layers.Dense(2)])]},
    'PRT_med_<-1>:64:2' : {'config':['PRT_median', -1, tf.keras.Sequential([tf.keras.layers.Dense(64, activation="relu"),tf.keras.layers.Dense(2)])]},
    #'PRT_med_<-1>:64(0.3):2' : {'config':['PRT_median', -1, tf.keras.Sequential([tf.keras.layers.Dense(64, activation="relu"),tf.keras.layers.Dropout(rate=0.3),tf.keras.layers.Dense(2)])]},
    'PRT_med_<-2>:<-1>:2' : {'config':['PRT_median', -2, tf.keras.Sequential([tf.keras.layers.Dense(2)])]},
    'PRT_med_:2' : {'config':['PRT_median', 0, tf.keras.Sequential([tf.keras.layers.Dense(2)])]},
    'PRT_sel10_<-1>:2' : {'config':['PRT_sel10', -1, tf.keras.Sequential([tf.keras.layers.Dense(2)])]},
    #'PRT_sel10_<-1>:32:2' : {'config':['PRT_sel10', -1, tf.keras.Sequential([tf.keras.layers.Dense(32, activation="relu"),tf.keras.layers.Dense(2)])]},
    'PRT_sel10_<-1>:64:2' : {'config':['PRT_sel10', -1, tf.keras.Sequential([tf.keras.layers.Dense(64, activation="relu"),tf.keras.layers.Dense(2)])]},
    #'PRT_sel10_<-1>:64(0.3):2' : {'config':['PRT_sel10', -1, tf.keras.Sequential([tf.keras.layers.Dense(64, activation="relu"),tf.keras.layers.Dropout(rate=0.3),tf.keras.layers.Dense(2)])]},
    'PRT_sel10_<-2>:<-1>:2' : {'config':['PRT_sel10', -2, tf.keras.Sequential([tf.keras.layers.Dense(2)])]},
    'PRT_sel10_:2' : {'config':['PRT_sel10', 0, tf.keras.Sequential([tf.keras.layers.Dense(2)])]},
}

In [ ]:
for cv in range(1,6):
    for label in models:
        BATCH_SIZE = 256
        # load train, validation and calibration data for the respective cross-validation split, and the test data
        TRAIN_DATAPATH = f'../data/PROSPECT_median_training{cv}.csv'
        rtdata_median = RetentionTimeDataset(data_source=TRAIN_DATAPATH,
                                      seq_length=30, batch_size=BATCH_SIZE, val_ratio=0., test=False,
                                      sequence_col='modified_sequence_single_letter',
                                      target_col='median')
        TRAIN_DATAPATH = f'../data/PROSPECT_sel10_training{cv}.csv'
        rtdata_sel10 = RetentionTimeDataset(data_source=TRAIN_DATAPATH,
                                      seq_length=30, batch_size=BATCH_SIZE, val_ratio=0., test=False,
                                      sequence_col='modified_sequence_single_letter',
                                      target_col='indexed_retention_time')
        VALIDATION_DATAPATH = f'../data/PROSPECT_median_validation{cv}.csv'
        validation_rtdata = RetentionTimeDataset(data_source=VALIDATION_DATAPATH,
                                                 seq_length=30, batch_size=BATCH_SIZE, val_ratio=1., test=False,
                                                 sequence_col='modified_sequence_single_letter',
                                                 target_col='median')
        CALIBRATION_DATAPATH = f'../data/PROSPECT_median_calibration{cv}.csv'
        calibration_rtdata = RetentionTimeDataset(data_source=CALIBRATION_DATAPATH,
                                           seq_length=30, batch_size=BATCH_SIZE, test=True,
                                           sequence_col='modified_sequence_single_letter',
                                           target_col='median')
        calibration_targets = calibration_rtdata.get_split_targets(split="test")
        TEST_DATAPATH = '../data/PROSPECT_median_holdout_cv.csv'
        test_rtdata = RetentionTimeDataset(data_source=TEST_DATAPATH,
                                           seq_length=30, batch_size=BATCH_SIZE, test=True,
                                           sequence_col='modified_sequence_single_letter',
                                           target_col='median')
        test_targets = test_rtdata.get_split_targets(split="test")
    
    
        print("\n#####", label, cv, '#####\n')
        models[label][cv] = {}
        model_class, frozen, output_layer = models[label]['config']
        output_dir = f"output_{label}/cv{cv}/"
        if model_class == 'PRT_median':
            model_save_path = f"./output_median/cv{cv}/best"
            rtdata = rtdata_median
            epochs = 50
        elif model_class == 'PRT_sel10':
            model_save_path = f"./output_sel10/cv{cv}/best"
            rtdata = rtdata_sel10
            epochs = 6
        
        # in case this notebook crashes, don't rerun experiments
        if os.path.exists(os.path.join(output_dir, "history.pkl")):
            continue
        set_global_seed(42)
        
        # load weights and define which layers are re-trained based on config
        qr_model = PrositRetentionTimePredictor(seq_length=30, vocab_dict=ALPHABET_MOD)
        if frozen:
            qr_model.load_weights(model_save_path)
            for layer in qr_model.layers[:frozen]:
                layer.trainable = False
        qr_model.output_layer = output_layer

        # setup a learning rate schedule
        train_steps = BATCH_SIZE * len(rtdata.train_data) * epochs
        lr_fn = tf.optimizers.schedules.PolynomialDecay(1e-3, train_steps, 1e-6, 2)
        opt = tf.optimizers.Adam(lr_fn)

        qr_model.compile(optimizer=opt, 
                  loss=QuantileLoss(tf.constant([[0.1, 0.9]])),
                  metrics=['mean_absolute_error', 
                           RelativeCentralDistance(),
                           IntervalConformalQuantile(alpha=0.1),
                           AbsoluteIntervalSize(),
                          ])
        # save best model based on validation loss
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(output_dir, 'best'),
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            save_best_only=True)
        history = qr_model.fit(rtdata.train_data,
                               validation_data=validation_rtdata.val_data,
                               callbacks=[model_checkpoint_callback],
                               epochs=epochs,
                               verbose=2)
        with open(os.path.join(output_dir,'history.pkl'), 'wb') as f:
            pickle.dump(history, f)

# Conformal Prediciton (median datasets)

After Quantile Regression, apply conformal Prediction with alpha = 0.1 to assure marginal coverage of 0.9 for each model.
The same is done for a randomized background model that is identical to the original model with respect to the heuristic interval sizes, but they are randomly reassociated betwee the test datapoints.

In [ ]:
def plot_conformal_scores(conf_scores, quantile=None, xlim=6):
    fig,ax = plt.subplots()
    xmax = conf_scores.std() * xlim
    ax.hist(conf_scores[conf_scores < xmax], bins=100)
    if quantile:
        ax.axvline(quantile, color='red', alpha=0.5)
    ax.set(xlabel='conformal score', ylabel='# per bin', title=f'conformal scores (<{xlim} std.dev. from mean)')
    plt.show()

In [ ]:
for cv in range(1,6):
    for label in models:
        output_dir = f"output_{label}/cv{cv}/"
        # skip experiments were quantile regression was not successful
        if not os.path.exists(os.path.join(output_dir, 'history.pkl')):
            continue
        
        # load calibration data for calculation of conformal scores
        CALIBRATION_DATAPATH = f'../data/PROSPECT_median_calibration{cv}.csv'
        calibration_rtdata = RetentionTimeDataset(data_source=CALIBRATION_DATAPATH,
                                           seq_length=30, batch_size=BATCH_SIZE, test=True,
                                           sequence_col='modified_sequence_single_letter',
                                           target_col='median')
        calibration_targets = calibration_rtdata.get_split_targets(split="test")
        
        # load best model from weights
        model_class, frozen, output_layer = models[label]['config']
        qr_model = PrositRetentionTimePredictor(seq_length=30, vocab_dict=ALPHABET_MOD)
        qr_model.output_layer = output_layer
        qr_model.load_weights(os.path.join(output_dir, "best"))
        models[label][cv]['model'] = qr_model

        # predict quantiles for calibration data, then calculate conformal scores and correct test-intervals accordingly
        quantile_predictions = qr_model.predict(calibration_rtdata.test_data)
        conf_scores = IntervalConformalScore(reduction='none')(np.expand_dims(calibration_targets,-1), quantile_predictions).numpy()
        conf_quantile = IntervalConformalQuantile(alpha=0.1, reduction='none')(np.expand_dims(calibration_targets,-1), quantile_predictions).numpy()
        intervals = qr_model.predict(test_rtdata.test_data)
        intervals[:,0] -= conf_quantile
        intervals[:,1] += conf_quantile

        # calculate some other stats
        interval_sizes = intervals[:,1] - intervals[:,0]
        within = (test_targets >= intervals[:,0]) & (test_targets <= intervals[:,1])
        pvalue = ks_2samp(interval_sizes[within], interval_sizes[~within]).pvalue # prob. for Null: distr are identical

        models[label][cv]['intervals'] = intervals
        models[label][cv]['conf_scores'] = conf_scores
        models[label][cv]['conf_quantile'] = conf_quantile
        models[label][cv]['within'] = within
        models[label][cv]['conf_scores_test'] = IntervalConformalScore(reduction='none')(np.expand_dims(test_targets,-1), qr_model.predict(test_rtdata.test_data)).numpy()
        models[label][cv]['pvalue'] = pvalue

        # plot conformal score distributions
        print('####', label, cv, '####\nconformal quantile:', conf_quantile, '\n')
        plot_conformal_scores(conf_scores, quantile=conf_quantile)
        
        # calculate a random background model
        quantile_predictions = qr_model.predict(calibration_rtdata.test_data)
        centers = (quantile_predictions[:,1] + quantile_predictions[:,0]) / 2
        half_iv = (quantile_predictions[:,1] - quantile_predictions[:,0]) / 2
        np.random.seed(42)
        p = np.random.permutation(quantile_predictions.shape[0])
        quantile_predictions = np.column_stack([centers - half_iv[p], centers + half_iv[p]])
        
        # perform conformalization for random background model
        conf_scores = IntervalConformalScore(reduction='none')(np.expand_dims(calibration_targets,-1), quantile_predictions).numpy()
        conf_quantile = IntervalConformalQuantile(alpha=0.1, reduction='none')(np.expand_dims(calibration_targets,-1), quantile_predictions).numpy()
        intervals = qr_model.predict(test_rtdata.test_data)
        centers = (intervals[:,1] + intervals[:,0]) / 2
        half_iv = (intervals[:,1] - intervals[:,0]) / 2
        np.random.seed(cv)
        p = np.random.permutation(intervals.shape[0])
        intervals = np.column_stack([centers - half_iv[p], centers + half_iv[p]])
        intervals[:,0] -= conf_quantile
        intervals[:,1] += conf_quantile

        interval_sizes = intervals[:,1] - intervals[:,0]
        within = (test_targets >= intervals[:,0]) & (test_targets <= intervals[:,1])
        models[label][cv]['rnd_conf_scores'] = conf_scores
        models[label][cv]['rnd_conf_quantile'] = conf_quantile
        models[label][cv]['rnd_intervals'] = intervals
        models[label][cv]['rnd_within'] = within

# Visualize some results

Plot interval size distributions for each individual model.

This section is optional and can be skipped.

In [ ]:
def plot_predictions_with_intervals(test_targets, test_estimates, intervals, label=None):
    fig,ax = plt.subplots()
    if label:
        ax.set_title(label)
    p = test_targets.argsort()
    ax.plot(test_targets[p], intervals[p, 0], alpha=0.1)
    ax.plot(test_targets[p], intervals[p, 1], alpha=0.1)
    ax.scatter(test_targets[p], test_estimates[p], s=1, alpha=0.1)
    ax.plot((test_targets.min(),test_targets.max()), (test_targets.min(),test_targets.max()), alpha=0.3, color='black', linestyle='--')
    ax.set(title=f'{label+" " if label else ""}predictions with error intervals, sorted by RT',
           xlabel='true retention time',
           ylabel='predicted retention time')
    plt.show()

In [ ]:
# plot interval size distributions
for label in models:
    for cv in range(1,6):
        output_dir = f"output_{label}/cv{cv}/"
        if not os.path.exists(os.path.join(output_dir, 'history.pkl')):
            continue
        intervals = models[label][cv]['intervals']
        print(intervals.shape)
        interval_sizes = intervals[:,1] - intervals[:,0]

        print('####', label, cv, '####')
        xlim = 6.
        fig,ax = plt.subplots()
        xmin = interval_sizes.mean() - interval_sizes.std() * xlim
        xmax = interval_sizes.mean() + interval_sizes.std() * xlim
        ax.hist(interval_sizes, bins=200, range=(0,60))#[(interval_sizes >= xmin) & (interval_sizes <= xmax)], bins=100)

        ax.set(xlim=(0,60), ylim=(0,15000))
        ax.set(xlabel='interval size', ylabel='# per bin', title=f'conformalized interval sizes')
        plt.show()

In [ ]:
for label in models:
    for cv in range(6):
        output_dir = f"output_{label}/cv{cv}/"
        if not os.path.exists(os.path.join(output_dir, 'history.pkl')):
            continue
        intervals = models[label][cv]['intervals']
        print(intervals.shape)
        interval_sizes = intervals[:,1] - intervals[:,0]
        within = (test_targets >= intervals[:,0]) & (test_targets <= intervals[:,1])

        print('####', label, cv, '####')

        pvalue = ks_2samp(interval_sizes[within], interval_sizes[~within]).pvalue # prob. for Null: distr are identical
        print(f"p = {pvalue:.5f} : {'Reject' if pvalue < 0.01 else 'Accept'} Null Hypothesis (Distr. identical)")

        xlim = 6.
        fig,ax = plt.subplots()
        xmin = interval_sizes.mean() - interval_sizes.std() * xlim
        xmax = interval_sizes.mean() + interval_sizes.std() * xlim
        ax.hist(interval_sizes[within], bins=100, range=(0,60), histtype='step', density=True, color='C0', label='inside interval')
        ax.hist(interval_sizes[~within], bins=100, range=(0,60), histtype='step', density=True, color='C1', label='outside interval')

        ax.set(xlim=(0,60), ylim=(0,0.42))
        ax.set(xlabel='interval size', ylabel='fraction per bin', title=f'PDF of conformalized interval sizes')
        ax.text(0.98, 0.8, f"identical: p = {pvalue:.5f}", transform=ax.transAxes, ha='right', va='top')
        ax.legend()
        plt.show()

# Conformal Prediciton (sel10 datasets)

Do conformal prediction, as above, but use sel10 calibration and test data (up to 10 raw iRT values per unique sequence) instead of median ones.

In [ ]:
TEST_DATAPATH = '../data/PROSPECT_sel10_holdout_cv.csv'
test_rtdata_sel10 = RetentionTimeDataset(data_source=TEST_DATAPATH,
                                   seq_length=30, batch_size=BATCH_SIZE, test=True,
                                   sequence_col='modified_sequence_single_letter',
                                   target_col='indexed_retention_time')
test_targets_sel10 = test_rtdata_sel10.get_split_targets(split="test")

# copy config from the "median" results dictionary
models_sel10 = {l:{'config':models[l]['config']} for l in models}

for cv in range(6):
    for label in models_sel10:
        output_dir = f"output_{label}/cv{cv}/"
        if not os.path.exists(os.path.join(output_dir, 'history.pkl')):
            continue
        models_sel10[label][cv] = {}
        
        # load calibration data for calculation of conformal scores
        CALIBRATION_DATAPATH = f'../data/PROSPECT_sel10_calibration{cv}.csv'
        calibration_rtdata = RetentionTimeDataset(data_source=CALIBRATION_DATAPATH,
                                           seq_length=30, batch_size=BATCH_SIZE, test=True,
                                           sequence_col='modified_sequence_single_letter',
                                           target_col='indexed_retention_time')
        calibration_targets = calibration_rtdata.get_split_targets(split="test")
        
        # load best model from weights
        model_class, frozen, output_layer = models_sel10[label]['config']
        qr_model = PrositRetentionTimePredictor(seq_length=30, vocab_dict=ALPHABET_MOD)
        qr_model.output_layer = output_layer
        qr_model.load_weights(os.path.join(output_dir, "best"))
        models_sel10[label][cv]['model'] = qr_model

        # predict quantiles for calibration data, then calculate conformal scores and correct test-intervals accordingly
        quantile_predictions = qr_model.predict(calibration_rtdata.test_data)
        conf_scores = IntervalConformalScore(reduction='none')(np.expand_dims(calibration_targets,-1), quantile_predictions).numpy()
        conf_quantile = IntervalConformalQuantile(alpha=0.1, reduction='none')(np.expand_dims(calibration_targets,-1), quantile_predictions).numpy()
        intervals = qr_model.predict(test_rtdata_sel10.test_data)
        intervals[:,0] -= conf_quantile
        intervals[:,1] += conf_quantile
        
        # calculate some other stats
        interval_sizes = intervals[:,1] - intervals[:,0]
        within = (test_targets_sel10 >= intervals[:,0]) & (test_targets_sel10 <= intervals[:,1])
        pvalue = ks_2samp(interval_sizes[within], interval_sizes[~within]).pvalue # prob. for Null: distr are identical

        models_sel10[label][cv]['intervals'] = intervals
        models_sel10[label][cv]['conf_scores'] = conf_scores
        models_sel10[label][cv]['conf_quantile'] = conf_quantile
        models_sel10[label][cv]['within'] = within
        models_sel10[label][cv]['conf_scores_test'] = IntervalConformalScore(reduction='none')(np.expand_dims(test_targets_sel10,-1), 
                                                                                               qr_model.predict(test_rtdata_sel10.test_data)).numpy()
        models_sel10[label][cv]['pvalue'] = pvalue

        print('####', label, cv, '####\nconformal quantile:', conf_quantile, '\n')
        plot_conformal_scores(conf_scores, quantile=conf_quantile)
        #plot_predictions_with_intervals(test_targets, new_predictions, intervals)
        
        # calculate a random background model
        quantile_predictions = qr_model.predict(calibration_rtdata.test_data)
        centers = (quantile_predictions[:,1] + quantile_predictions[:,0]) / 2
        half_iv = (quantile_predictions[:,1] - quantile_predictions[:,0]) / 2
        np.random.seed(42)
        p = np.random.permutation(quantile_predictions.shape[0])
        quantile_predictions = np.column_stack([centers - half_iv[p], centers + half_iv[p]])
        
        # perform conformalization for random background model
        conf_scores = IntervalConformalScore(reduction='none')(np.expand_dims(calibration_targets,-1), quantile_predictions).numpy()
        conf_quantile = IntervalConformalQuantile(alpha=0.1, reduction='none')(np.expand_dims(calibration_targets,-1), quantile_predictions).numpy()
        intervals = qr_model.predict(test_rtdata_sel10.test_data)
        centers = (intervals[:,1] + intervals[:,0]) / 2
        half_iv = (intervals[:,1] - intervals[:,0]) / 2
        np.random.seed(cv)
        p = np.random.permutation(intervals.shape[0])
        intervals = np.column_stack([centers - half_iv[p], centers + half_iv[p]])
        intervals[:,0] -= conf_quantile
        intervals[:,1] += conf_quantile

        interval_sizes = intervals[:,1] - intervals[:,0]
        within = (test_targets_sel10 >= intervals[:,0]) & (test_targets_sel10 <= intervals[:,1])
        models_sel10[label][cv]['rnd_conf_scores'] = conf_scores
        models_sel10[label][cv]['rnd_conf_quantile'] = conf_quantile
        models_sel10[label][cv]['rnd_intervals'] = intervals
        models_sel10[label][cv]['rnd_within'] = within
        print('####', label, cv, 'rnd', '####\nconformal quantile:', conf_quantile, '\n')
        plot_conformal_scores(conf_scores, quantile=conf_quantile)

# Save results as pickle files

In [ ]:
# delete all TF objects (models, config and histories) from dict. 
# For whatever reason, they cannot be loaded correctly when pickled
for label in models:
    try:
        del models[label]['config']
    except:
        pass
    for cv in range(6):
        try:
            del models[label][cv]['model']
            del models[label][cv]['history']
        except:
            pass
for label in models:
    try:
        del models_sel10[label]['config']
    except:
        pass
    for cv in range(6):
        try:
            del models_sel10[label][cv]['model']
            del models_sel10[label][cv]['history']
        except:
            pass

In [ ]:
with open('../data/QR_results_cv_with_rnd.pkl', 'wb') as f:
    pickle.dump(models, f)

In [ ]:
with open('../data/QR_results_sel10_cv_with_rnd.pkl', 'wb') as f:
    pickle.dump(models_sel10, f)